Coche une seule bonne réponse par question.

📊 1–4 : Data Analysis
1. Que renvoie la méthode .groupby() dans pandas ?

A. Une nouvelle DataFrame triée

B. Un tableau de corrélation

C. Un objet regroupé par clé ✅

D. Une liste d’index

2. Quelle méthode calcule la médiane d’une colonne ?

A. .mean()

B. .median() ✅

C. .mod()

D. .mode()

3. Quel type de graphique est idéal pour analyser une distribution ?

A. Barplot

B. Boxplot ✅

C. Line plot

D. Heatmap

4. Quel code remplace les NaN dans une colonne par 0 ?

A. df.fillna(0) ✅

B. df.dropna(0)

C. df.isna(0)

D. df.remove(0)


🤖 5–8 : Machine Learning

5. Quel algorithme est non-linéaire ?

A. Linear Regression

B. Logistic Regression

C. Decision Tree ✅

D. Ridge Regression

6. Pourquoi utilise-t-on un split train/test ?

A. Pour nettoyer les données

B. Pour évaluer la généralisation du modèle ✅

C. Pour encoder les labels

D. Pour normaliser les colonnes

7. Quelle métrique est la plus adaptée si les classes sont déséquilibrées ?

A. Accuracy

B. Precision

C. Recall

D. F1-score ✅

8. Que fait le paramètre max_depth dans un arbre ?

A. Change la taille du dataset

B. Limite la profondeur de l’arbre ✅

C. Fixe le nombre de features

D. Supprime les valeurs aberrantes


🖼️ 9–12 : Computer Vision

9. Quelle est la taille des images dans le dataset MNIST ?

A. 16x16

B. 32x32

C. 28x28 ✅

D. 64x64

10. Quelle bibliothèque est utilisée pour charger des images en OpenCV ?

A. pandas

B. cv2 ✅

C. matplotlib

D. sklearn

11. Que fait une convolution dans un CNN ?

A. Elle agrandit l’image

B. Elle réduit la taille de l’image

C. Elle extrait des motifs ✅

D. Elle encode l’image

12. Quelle couche suit généralement une couche convolution ?

A. Dense

B. Embedding

C. Dropout

D. MaxPooling ✅

🧠 13–16 : Deep Learning / NLP

13. Que retourne une couche Embedding ?

A. Une image

B. Une probabilité

C. Un vecteur dense ✅

D. Un token

14. Quelle est la fonction d’activation la plus utilisée en NLP ?

A. ReLU

B. Sigmoid

C. Softmax ✅

D. Tanh

15. Que permet le padding dans les séquences NLP ?

A. Normaliser les vecteurs

B. Supprimer les tokens inutiles

C. Uniformiser la longueur des phrases ✅

D. Compresser les données

16. Quelle fonction compile un modèle Keras ?

A. model.fit()

B. model.compile() ✅

C. model.evaluate()

D. model.predict()


📚 17–20 : LLM / Transformers

17. Que veut dire “LLM” ?

A. Long Language Model

B. Linear Language Mapping

C. Large Language Model ✅

D. Learned Lexical Model

18. Quel est le rôle du tokenizer ?

A. Entraîner un modèle

B. Compiler les données

C. Convertir du texte en tokens ✅

D. Prédire les classes

19. Quel modèle est auto-régressif ?

A. BERT

B. RoBERTa

C. GPT ✅

D. T5

20. Quelle bibliothèque gère les LLMs open source ?

A. scikit-learn

B. openai

C. transformers ✅

D. keras.


**Projet 4 — OCR intelligent pour lecture automatique de factures**

Objectif : Extraire texte structuré (nom, montant, date) à partir d’une image PDF ou JPEG

Techniques : Tesseract OCR, OpenCV, post-processing NLP

📦 Dataset :
Invoice OCR Dataset

🔧 Tâches :

•	Extraction texte avec Tesseract

•	Nettoyage regex

•	Classification des champs (entité nommée)

•	Export JSON structuré



**Tesseract** est un moteur OCR open-source développé par Google.
Son but est simple : convertir une image (ou PDF scanné) en texte brut.

🔹Traitement interne :

🔸Binarisation (noir/blanc)

🔸Détection des lignes de texte

🔸Segmentation des caractères

🔸Reconnaissance des caractères via des modèles ML pré-entraînés.

**OpenCV** (Open Computer Vision) sert à préparer et améliorer l’image avant de l’envoyer à Tesseract.
C’est la partie pré-traitement visuel, essentielle pour avoir un OCR précis.

🔹 Techniques courantes :

🔸 Conversion en niveaux de gris : enlève la couleur inutile.

🔸 Seuil (thresholding) : rend le texte plus contrasté.

🔸 Filtrage de bruit (blur, morphology) : supprime les points, taches.

🔸 Rotation / redressement : corrige une image penchée.

🔸 Découpage (ROI) : extraire une zone spécifique (ex: montant)

Une fois que Tesseract donne du texte brut, il faut le comprendre et le structurer.
 C’est le travail du traitement du langage naturel (**NLP**).

🔹 Objectifs :

🔸Extraire les informations structurées :
Nom de société ,
Date,
Montant total

🔸Corriger les erreurs d’OCR (ex: “T0tal” → “Total”).

🔸Détecter les entités nommées à l’aide de regex ou modèles NLP.


In [1]:
pip install pytesseract pillow opencv-python

In [4]:
import pytesseract
import cv2
import re
import json
import glob
import os
from PIL import Image

# 1. Configuration (Mettez le chemin de l'exécutable Tesseract si non dans le PATH)
# Exemple pour Windows : pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def preprocess_image(image_path):
    """
    Prétraitement d'image avec OpenCV pour améliorer l'exactitude de l'OCR.
    """
    try:
        # Charger l'image
        img = cv2.imread(image_path)
        if img is None:
            print(f"⚠️ Erreur: Impossible de lire l'image {image_path}. (Peut-être un PDF non géré par cv2 directement)")
            return None

        # Convertir en niveaux de gris
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Appliquer un seuillage (texte noir sur fond blanc)
        thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)[1]

        # Enregistrer l'image prétraitée temporairement (utile pour le debug)
        # Utiliser un nom de fichier temporaire unique pour les traitements parallèles si besoin,
        # ici on réutilise pour la simplicité.
        temp_path = "temp_preprocessed.png"
        cv2.imwrite(temp_path, thresh)

        return temp_path

    except Exception as e:
        print(f"Erreur lors du prétraitement de {image_path}: {e}")
        return image_path # Retourne le chemin original en cas d'échec

def extract_text_tesseract(image_path):
    """
    Extraction de texte brut à partir de l'image.
    """
    if image_path is None:
        return ""

    try:
        # Configuration pour l'extraction de documents (psm 6)
        text = pytesseract.image_to_string(Image.open(image_path), lang='eng', config='--psm 6')
        return text
    except Exception as e:
        print(f"Erreur Tesseract sur {image_path}: {e}")
        return ""


def clean_and_classify_fields(raw_text, filename):
    """
    Nettoyage (regex) et Classification des Champs (Entité Nommée) avec améliorations.
    """
    extracted_data = {'file_name': filename}

    # 1. Extraction du Nom du Vendeur (Amélioration)

    lines = raw_text.split('\n')
    # Les 5 premières lignes non vides
    header_lines = [line.strip() for line in lines if line.strip()][:5]

    seller_name_candidate = None
    if header_lines:
        # Tentative d'heuristique simple : prendre la ligne la plus longue du début comme nom
        potential_names = [line for line in header_lines if len(line.split()) >= 2]
        if potential_names:
            seller_name_candidate = max(potential_names, key=len)
            # Nettoyage supplémentaire
            seller_name_candidate = re.sub(r'(Tel|Fax|E-mail|Web|http|www)[^\n]*', '', seller_name_candidate, flags=re.IGNORECASE).strip()

    extracted_data['seller_name'] = seller_name_candidate

    # 2. Extraction du Montant Total (Amélioration)

    # Date et Numéro de Facture (inchangé)
    date_pattern = r"(?:Date of issue|Date\s*d'émission|Date):\s*(\d{2}[/.-]\d{2}[/.-]\d{4})"
    invoice_no_pattern = r"(?:Invoice no|Facture n°|N°\s*Facture|Ref):?\s*(\w+)"

    match_inv_no = re.search(invoice_no_pattern, raw_text, re.IGNORECASE)
    extracted_data['invoice_number'] = match_inv_no.group(1).strip() if match_inv_no else None

    match_date = re.search(date_pattern, raw_text, re.IGNORECASE)
    extracted_data['date_of_issue'] = match_date.group(1).strip() if match_date else None

    # Nouvelle Regex pour le montant total, supporte le point ou la virgule, et le symbole d'euro/dollar
    # Cherche une ligne qui contient "TOTAL" et capture le nombre.
    total_pattern = r"(?:TOTAL|MONTANT\s*TOTAL|Amount\s*Due|Net\s*to\s*Pay)\s*[:\s]*[\$€]?\s*([\d\s,]+\.\d{2}|[\d\s\.]+\,\d{2})"

    match_total = re.search(total_pattern, raw_text, re.IGNORECASE | re.MULTILINE)

    if match_total:
        total_amount = match_total.group(1).strip()

        # Nettoyage : retirer les espaces (séparateurs de milliers) et remplacer la virgule par un point
        total_amount = total_amount.replace(' ', '').replace(',', '.')

        try:
            extracted_data['gross_total_amount'] = float(total_amount)
        except ValueError:
            extracted_data['gross_total_amount'] = None
            print(f"Échec de la conversion du montant '{total_amount}' en nombre.")
    else:
        extracted_data['gross_total_amount'] = None

    # Si la regex ci-dessus échoue, essayons l'approche heuristique du dernier montant
    if extracted_data['gross_total_amount'] is None:
         # Chercher tous les montants qui ressemblent à des prix (format .XX)
        all_price_candidates = re.findall(r'(\d[\d\s,]*[\.\,]\d{2})(?:\s*€|\s*\$|\s*HT|\s*TTC)?', raw_text)
        if all_price_candidates:
            # On prend le dernier candidat de la liste (souvent le total)
            last_candidate = all_price_candidates[-1].replace(' ', '').replace(',', '.')
            try:
                extracted_data['gross_total_amount'] = float(last_candidate)
            except ValueError:
                pass

    return extracted_data

def process_folder(folder_path, output_filename="all_invoices_data.json"):
    """
    Traite toutes les images d'un dossier et exporte un fichier JSON consolidé.
    """

    # Liste des extensions de fichiers à traiter
    image_extensions = ['jpg', 'jpeg', 'png']

    all_invoices_data = []

    print(f"1. Démarrage du traitement par lot dans le dossier: {folder_path}")

    # Utilisation de glob pour trouver tous les fichiers avec les extensions spécifiées
    all_files = []
    for ext in image_extensions:
        all_files.extend(glob.glob(os.path.join(folder_path, f'*.{ext}')))

    if not all_files:
        print(f" Aucun fichier trouvé dans {folder_path}. Vérifiez le chemin et les extensions.")
        return

    for i, file_path in enumerate(all_files, 1):
        file_name = os.path.basename(file_path)
        print(f"\n--- Traitement {i}/{len(all_files)}: {file_name} ---")

        # 2. Prétraitement
        preprocessed_path = preprocess_image(file_path)

        # 3. Extraction du texte brut
        raw_text = extract_text_tesseract(preprocessed_path)

        # 4. Classification des Champs (Entité Nommée) par Regex
        structured_data = clean_and_classify_fields(raw_text, file_name)

        all_invoices_data.append(structured_data)

        # Nettoyage du fichier temporaire
        if preprocessed_path and os.path.exists(preprocessed_path):
            try:
                os.remove(preprocessed_path)
            except OSError as e:
                print(f"Impossible de supprimer le fichier temporaire: {e}")

    # 5. Export JSON Structuré Consolidé
    print("\n--------------------------------------")
    print("5. Compilation et exportation des données...")
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(all_invoices_data, f, ensure_ascii=False, indent=4)

    print(f"\n Traitement terminé. {len(all_invoices_data)} factures traitées.")
    print(f"Fichier JSON consolidé exporté sous : {output_filename}")


# --- Bloc Principal d'Exécution ---
if __name__ == "__main__":

    # ⚠️ REMPLACER 'path/to/your/dataset' PAR LE VRAI CHEMIN DE VOTRE DOSSIER D'IMAGES
    # Exemple : "C:/Users/User/Desktop/Invoice OCR Dataset/images"
    # ou "./dataset/images" si le dossier est dans le même répertoire que le script.

    invoice_folder_path = "/content/drive/MyDrive/ai/batch1_1"

    # Assurez-vous que le chemin d'accès au dossier est correct !
    if not os.path.isdir(invoice_folder_path):
        print(f" Le dossier spécifié n'existe pas : {invoice_folder_path}")
        print("Veuillez modifier la variable 'invoice_folder_path' dans le script.")
    else:
        process_folder(invoice_folder_path)

1. Démarrage du traitement par lot dans le dossier: /content/drive/MyDrive/ai/batch1_1

--- Traitement 1/500: batch1-0027.jpg ---

--- Traitement 2/500: batch1-0011.jpg ---

--- Traitement 3/500: batch1-0044.jpg ---

--- Traitement 4/500: batch1-0056.jpg ---

--- Traitement 5/500: batch1-0039.jpg ---

--- Traitement 6/500: batch1-0022.jpg ---

--- Traitement 7/500: batch1-0050.jpg ---

--- Traitement 8/500: batch1-0063.jpg ---

--- Traitement 9/500: batch1-0042.jpg ---

--- Traitement 10/500: batch1-0074.jpg ---

--- Traitement 11/500: batch1-0012.jpg ---

--- Traitement 12/500: batch1-0062.jpg ---

--- Traitement 13/500: batch1-0028.jpg ---

--- Traitement 14/500: batch1-0029.jpg ---

--- Traitement 15/500: batch1-0043.jpg ---

--- Traitement 16/500: batch1-0053.jpg ---

--- Traitement 17/500: batch1-0001.jpg ---

--- Traitement 18/500: batch1-0047.jpg ---

--- Traitement 19/500: batch1-0004.jpg ---

--- Traitement 20/500: batch1-0075.jpg ---

--- Traitement 21/500: batch1-0016.jpg --